In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor


In [9]:
!wget ds1-trees-ensemble.zip

--2019-03-18 12:41:57--  http://ds1-trees-ensemble.zip/
Resolving ds1-trees-ensemble.zip (ds1-trees-ensemble.zip)... 198.105.244.114, 198.105.254.114
Connecting to ds1-trees-ensemble.zip (ds1-trees-ensemble.zip)|198.105.244.114|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'index.html.1'

     0K                                                        1.44M=0s

2019-03-18 12:41:58 (1.44 MB/s) - 'index.html.1' saved [291]



In [ ]:
# Create csv for submission to kaggle

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['charged_off'] = estimator.predict_proba(X_test)[:, 1]
submission.to_csv('submission-001.csv', index=False)